## OkCupid DataSet

### Meeting_3, ...


### Recap meeting_1 decisions:

<ol>
  <p>Meeting 1, 16-10-2019</p>
    <li>Merge all the essays as a unique text excluding essay4, 5, 6. </li>
  <li>Plot distribution of avarage number of words per sentence.</li>
  <li>Dont apply lowercase() and expand_contractions() instead count the number of them.</li>
  

</ol>


### Recap meeting_2 decisions:

<ol>
  <p>Meeting 2,12-11-2019</p>
  <li>Re-code education feature</li>
  <li>Treating with outliers? keep or remove? min and max threshold?</li>
  <li>Incorrect or irrelevant information such as age 109 or sentences with length 1</li>
  <li>Check list of stopwords</li>

</ol>


### To discuss:

<ol>
  <p></p>
  <li>Feature extraction/ vectorization</li>
  <li>Feature reduction??? stemming?</li>
  <li>In isced column, do you what is None value? do you want to remove it from dataset?</li>
  

</ol>


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


### Load dataset 

In [5]:
#  Load dataset 
df = pd.read_csv('../../data/processed/preprocessed_cupid.csv', usecols=['age', 'sex','#anwps', 'clean_text', 'isced'])
print(df.shape)
df = df = df.dropna(subset=['clean_text'])
print(df.shape)

(51595, 5)
(51575, 5)


In [6]:
corpus = df['clean_text'].dropna()
target = df["isced"]


<p>Document-term matrix</p>
<li>Each document is represented as the multiset of the tokens that compose it </li>
<li>The value for each word position in the vector is its count</li>

<img src="rep2_image/vector_matrix.jpg">


In [7]:
# frequency encoding scikit-learn
vectorizer = CountVectorizer(binary=False, ngram_range=(1, 2))
vectors = vectorizer.fit_transform(corpus)


## Supervised classification: phases and datasets
There are two phases, the training phase and the inference phase
<ol>
  <p></p>
    <img src="rep2_image/split_dataset.JPG">
    
  
  

</ol>

In [8]:
X_train, X_val, y_train, y_val = train_test_split(vectors, target, train_size=0.75,
                                                          test_size=0.25, random_state = 0)

Questions that should be answered in training phase:
1. what are the features?
   * By far the most common class of features are words
   * Handeling commonly- occuring words: stopwords
   * Normalization: Make lower case vs. leave as-is
   * Stemming/ Lemmatization
2. what is the classification model/ algorithm?
   * Unordered One
   * Unordered Two
   * Unordered Three
3. what are the model parameters?



<p>Questions that should be answered in inference phase:</p>
<li>what is the expected performance? waht is a good measure? </li>



### Naive Bayes classifier
1. Naive Bayes is a probabilistic model
   
2. Naive, because it assumes features are independent of each other, given the class label
   
3. for text classification problems, naive Bayes models typically prvide very strong baselines


In [46]:
naive_model = MultinomialNB()
naive_model.fit(X_train, y_train)
gnb_predictions = naive_model.predict(X_val)
print("Final Accuracy for NB: %s"% accuracy_score(y_val, gnb_predictions))

Final Accuracy for NB: 0.506514657980456
